# Dask DataFrame

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "Dask DataFrame"
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.memory_usage_per_partition.html#dask.dataframe.DataFrame.memory_usage_per_partition
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.map_partitions.html#dask.dataframe.DataFrame.map_partitions
* https://docs.dask.org/en/stable/generated/dask.array.histogram.html
* https://docs.dask.org/en/stable/dataframe-categoricals.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.pivot_table.html#dask.dataframe.DataFrame.pivot_table
* https://docs.dask.org/en/stable/generated/dask.dataframe.groupby.DataFrameGroupBy.apply.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.Series.map_overlap.html#dask.dataframe.Series.map_overlap

* https://docs.dask.org/en/stable/best-practices.html
* https://docs.dask.org/en/stable/dashboard.html
* https://distributed.dask.org/en/stable/client.html

## Задачи для совместного разбора

In [1]:
import dask

dask.__version__

'2022.12.0'

In [2]:
# !pip install --user --no-warn-script-location dask==2022.12.0

In [3]:
# !pip install --user distributed==2022.11.1
# !pip install --user bokeh==2.4.2

1\. Создать `dask.DataFrame`. Рассмотреть основные возможности (выбор строк, группировки, работа с датами, добавление столбцов)

In [4]:
df = dask.datasets.timeseries(start='2000-01-01', end='2003-10-10')
df

,name,id,x,y
npartitions=1378,,,,
2000-01-01,object,int32,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2003-10-09,...,...,...,...
2003-10-10,...,...,...,...


In [5]:
df["xy"] = df["x"] + df["y"]
df.head(5)

,name,id,x,y,xy
timestamp,,,,,
2000-01-01 00:00:00,Laura,992,0.175001,0.063634,0.238635
2000-01-01 00:00:01,Wendy,942,-0.845097,0.485326,-0.359771
2000-01-01 00:00:02,Ray,1020,-0.017909,-0.563075,-0.580983
2000-01-01 00:00:03,Tim,967,0.823212,-0.323800,0.499412
2000-01-01 00:00:04,Tim,998,-0.601332,0.409343,-0.191989


In [6]:
df[df["xy"] >= df["x"].mean()].head(5)

,name,id,x,y,xy
timestamp,,,,,
2000-01-01 00:00:00,Laura,992,0.175001,0.063634,0.238635
2000-01-01 00:00:03,Tim,967,0.823212,-0.323800,0.499412
2000-01-01 00:00:05,Tim,977,0.513431,-0.471632,0.041799
2000-01-01 00:00:06,Ray,1011,0.628083,-0.450230,0.177853
2000-01-01 00:00:07,Frank,1013,0.197694,0.987872,1.185567


In [ ]:
%%time
df.groupby(df.index.dt.to_period("M"))["x"].size().compute()

2\. Рассмотреть средства диагностики `dask`

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
%%time
with ProgressBar():
    df.groupby(df.index.dt.to_period("M"))["x"].size().compute()

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
%%time
df.groupby(df.index.dt.to_period("M"))["x"].size().compute()

3\. Изучить, как влияет вызов `persist` на производительность

In [ ]:
female = {"Edith", "Wendy", "Quinn", "Hannah", "Ursula", "Laura", "Sarah", "Alice"}

female_df = df[df["name"].isin(female)].persist()
female_df

In [ ]:
%%time
female_df.compute()

In [ ]:
%%time
female_df.groupby("name")["x"].mean().compute()

In [ ]:
%%time
(female_df["y"] * 10).mean().compute()

## Лабораторная работа 13

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.DataFrame` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.DataFrame` к `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.DataFrame` (то есть то, что было получено до вызова `compute`, а не после).

1\. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты. Выведите на экран информацию о количестве сегментов и типе столбцов. Выведите на экран 5 первых и 5 последних строк таблицы. В случае сообщения об ошибке объясните причину и исправьте ошибку.

#### ValueError: Mismatched dtypes found in `pd.read_csv`.
#### Это происходит из-за сбоя вывода dash's type. (Пропущенные значения не правильно интерпретируются) 
#### С помощью `assume_missing=True`, предполагается, что все целочисленные столбцы, которые не указаны в dtype, содержат пропущенные значения и преобразуются в значения с плавающей точкой (float64). 

In [2]:
import dask.dataframe as dd

In [3]:
df = dd.read_csv("./13_dask_dataframe_data/recipes_full/recipes_full_*.csv",
                 parse_dates=['submitted'],
                 assume_missing=True)
df.head(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970.0,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9.0
1,1089012.0,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5.0
2,1428572.0,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5.0
3,1400250.0,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1.0
4,387709.0,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10.0


In [4]:
df.tail(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
278949,1029131.0,tuti waffle snackies steakhouse,19.0,171345.0,1973-10-18,4.0,"according to a providence journal article, ama...",4.0
278950,1700703.0,noelias cheats crocante fleisch zitumbuwa,1.0,30228.0,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1.0
278951,1910650.0,rubbed restuffed pelmeni bedouin flavourful,60.0,591905.0,2009-09-26,3.0,another great recipe to add to the growing swe...,2.0
278952,713836.0,stems polpettine peezi,NaN,357389.0,2003-09-30,4.0,adapted from top secret recipes. love this!,9.0
278953,660699.0,clementines,64.0,29196.0,1973-06-03,6.0,this would make a great start to your holiday ...,8.0


2\. Выведите на экран следующую информацию:
* количество сегментов в таблице
* объем используемой памяти каждого сегмента
* количество строк в каждом сегменте
* количество строк во всей таблице.

In [5]:
df.npartitions

8

In [6]:
df.memory_usage_per_partition().compute()

0    17853248
1    17853248
2    17853248
3    17853248
4    17853248
5    17853184
6    17853184
7    17853184
dtype: int64

In [7]:
df.map_partitions(len).compute()

0    278955
1    278955
2    278955
3    278955
4    278955
5    278954
6    278954
7    278954
dtype: int64

In [8]:
df.shape[0].compute()

2231637

3\. В архиве `recipes_additional.zip` находятся файлы, содержащие информацию об рецептах блюд, полученные из другого источника. В связи с этим названия и количество столбцов в этих данных отличаются от того, что находится в архиве `recipes_full.zip`.

Объедините два набора набора данных, согласовав названия столбцов и форматы данных. Добавьте столбец с информацией о том, из какого датасета была получены данные.

In [9]:
df['id'] = df.id.astype(int).astype(str)
# df['contributor_id'] = df.contributor_id.astype(int)
df['source'] = 'recipes_full'
df = df.rename(columns={'submitted': 'date_of_publication'})

In [10]:
df_2 = dd.read_csv("./13_dask_dataframe_data/recipes_additional/recipes_additional_*.csv",
                 assume_missing=True)
df_2['date_of_publication'] = dd.to_datetime(df_2['date_of_publication'], format='%d/%m/%Y')
df_2['hours'] = df_2['hours'] * 60
df_2['source'] = 'recipes_additional'
df_2 = df_2.rename(columns={'hours': 'minutes', '#ingredients': 'n_ingredients', 'recipe_id': 'id'})

In [11]:
df_2.tail(5)

,name,description,minutes,date_of_publication,#tags,n_ingredients,id,views_30_days,source
325814,plas hachis,"if you like the sound of this, you are gonna l...",35.0,1997-06-29,5.0,1.0,20eadf,667.0,recipes_additional
325815,stephen jackpot guatemalan bredie cocoatana ma...,a cool and creamy dessert!,42.0,1981-09-16,10.0,9.0,14d0d8,2269.0,recipes_additional
325816,derves camp schmere krbissuppe fldekager,quick and easy. tasty alternative to tradition...,15.0,1999-01-22,5.0,4.0,1c0728,681.0,recipes_additional
325817,texans zuring mumble limas bouclair besan duvall,5-ingredient fix contest entry. tasty little ...,59.0,1973-02-07,9.0,5.0,eb528,3683.0,recipes_additional
325818,voi buying soutzoukakia mona biskvi uukkous tr...,this sounds tasty and quick and easy to make. ...,52.0,2009-08-16,2.0,9.0,6d753,1244.0,recipes_additional


In [12]:
df.tail(5)

,id,name,minutes,contributor_id,date_of_publication,n_steps,description,n_ingredients,source
278949,1029131,tuti waffle snackies steakhouse,19.0,171345.0,1973-10-18,4.0,"according to a providence journal article, ama...",4.0,recipes_full
278950,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228.0,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1.0,recipes_full
278951,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905.0,2009-09-26,3.0,another great recipe to add to the growing swe...,2.0,recipes_full
278952,713836,stems polpettine peezi,NaN,357389.0,2003-09-30,4.0,adapted from top secret recipes. love this!,9.0,recipes_full
278953,660699,clementines,64.0,29196.0,1973-06-03,6.0,this would make a great start to your holiday ...,8.0,recipes_full


#### Объедините два набора набора данных, согласовав названия столбцов и форматы данных. Добавьте столбец с информацией о том, из какого датасета была получены данные.

In [13]:
result_df = dd.concat([df, df_2], ignore_index=True, sort=False)
result_df.head(5)

,id,name,minutes,contributor_id,date_of_publication,n_steps,description,n_ingredients,source,#tags,views_30_days
0,683970,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9.0,recipes_full,NaN,NaN
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5.0,recipes_full,NaN,NaN
2,1428572,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5.0,recipes_full,NaN,NaN
3,1400250,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1.0,recipes_full,NaN,NaN
4,387709,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10.0,recipes_full,NaN,NaN


In [14]:
result_df.tail(5)

,id,name,minutes,contributor_id,date_of_publication,n_steps,description,n_ingredients,source,#tags,views_30_days
325814,20eadf,plas hachis,35.0,NaN,1997-06-29,NaN,"if you like the sound of this, you are gonna l...",1.0,recipes_additional,5.0,667.0
325815,14d0d8,stephen jackpot guatemalan bredie cocoatana ma...,42.0,NaN,1981-09-16,NaN,a cool and creamy dessert!,9.0,recipes_additional,10.0,2269.0
325816,1c0728,derves camp schmere krbissuppe fldekager,15.0,NaN,1999-01-22,NaN,quick and easy. tasty alternative to tradition...,4.0,recipes_additional,5.0,681.0
325817,eb528,texans zuring mumble limas bouclair besan duvall,59.0,NaN,1973-02-07,NaN,5-ingredient fix contest entry. tasty little ...,5.0,recipes_additional,9.0,3683.0
325818,6d753,voi buying soutzoukakia mona biskvi uukkous tr...,52.0,NaN,2009-08-16,NaN,this sounds tasty and quick and easy to make. ...,9.0,recipes_additional,2.0,1244.0


4\. Выясните, сколько рецептов содержат слово `chocolate`. Выведите на экран набор уникальных идентификаторов авторов таких рецептов.

In [15]:
result_df[result_df.name == 'chocolate'].head(1000)

C:\Users\Артём\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:7722: UserWarning: Insufficient elements for `head`. 1000 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,id,name,minutes,contributor_id,date_of_publication,n_steps,description,n_ingredients,source,#tags,views_30_days
133279,1341834,chocolate,44.0,37779.0,1986-10-14,2.0,"growing up in kearney, missouri in the '60s & ...",2.0,recipes_full,NaN,NaN


In [16]:
result_df[result_df.name == 'chocolate'].compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   ['n_steps']
  Missing: []

In [ ]:
result_df = result_df.n_steps.astype('float64')

In [25]:
type(result_df.n_steps.tail(1).values[0])

numpy.float64

In [28]:
result_df['#tags'].max().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   ['n_steps']
  Missing: []

5\. При помощи функции `da.histogram` посчитайте значения для построения гистограммы для столбца `n_ingredients`. Визуализируйте полученные результаты при помощи функции `plt.bar`. Добавьте на рисунок 2 вертикальные линии, соответствующие квантилям уровней 0.25 и 0.75. Сделайте масштаб вертикальной оси логарифмическим.

Допускается вычисление статистики по столбцу (применение метода `compute`) до того, как будет вызван `da.histogram`. 

6\. Постройте сводную таблицу при помощи метода `pivot_table`, где по строкам располагаются года, по столбцам - названия месяцев, а в ячейках содержится средняя длина рецептов в данный год и месяц.

7\. Удалите строки, которые содержат пропуске в столбце `contributor_id` и приведите его тип к целочисленному. Для каждого `contributor_id` найдите топ-5 слов, которых этот пользователь наиболее часто использовал в названиях своих рецептов. Выведите на экран статистику для пользователя с ID 1530.

8\. В файлах архива `site_logs.zip` находятся логи системы. Считайте эти логи в виде `dask.bag`, выделите их них дату, время, id службы, уровень лога и сообщение. Объедините дату и время в одну строку. Преобразуйте `dask.bag` в `dask.dataframe`, предварительно оставив только логи уровня "INFO". Подтвердите корректность решения, выведя на экран уникальные значения столбца `level` полученной таблицы.

```
date(ymd)   time(HMS)   service_id level   msg
081110      103041      34         INFO    dfs.FSNamesystem: BLOCK* NameSystem.delete: ...
```

9\. Добавьте в таблицу из задания 8 столбец `is_delete_op`, который содержит флаг: является ли данная операция удалением чего-либо или нет. Для выяснения этого факта проанализизируйте сообщение лога. 

Для каждого лога подсчитайте, сколько из записанных перед ним последних 100 логов являются операциями удаления. Для вычислений используйте метод `map_overlap`. Посчитайте и выведите на экран, сколько существует подпоследовательностей не менее чем из 75 сообщений, содержащих операции удаления.

10\. Сгруппируйте таблицу с логами по часу, в котором был оставлен лог. Под часом имеется в виду час конкретного дня, то связь связка год-месяц-день-час.  Посчитайте, сколько логов было записано за каждый час. Визуализируйте полученную динамику. Добавьте подписи засечек, подписи осей и название рисунка.